In [41]:
# Add API Keys here
GROQ_API_KEY_i_1 = "..."
GROQ_API_KEY_i_2 = "..."

In [1]:
!pip install pathway sentence-transformers pandas numpy tqdm openai groq python-dotenv -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Verify the path exists
import os
if os.path.exists("/content/drive/MyDrive/KDSH26/Dataset/"):
    print("✅ Dataset found on Drive!")
else:
    print("❌ Path not found. Please check your Drive folder structure.")

Mounted at /content/drive
✅ Dataset found on Drive!


In [42]:
# ==========================================
# FILE 1: DATA INGESTION
# ==========================================
%%writefile data_ingestion.py
import pathway as pw
from sentence_transformers import SentenceTransformer
import os
from typing import Any

# Define Embedding Model Globally
EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'
GLOBAL_EMBEDDING_MODEL = SentenceTransformer(EMBEDDING_MODEL_NAME)
CHUNK_SIZE = 2048
OVERLAP = 256

class NovelIngestionPipeline:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.model = GLOBAL_EMBEDDING_MODEL

    @staticmethod
    def _split_text_sliding_window(text: str, chunk_size=CHUNK_SIZE, overlap=OVERLAP):
        words = text.split()
        n_words = len(words)
        chunks = []
        step = chunk_size - overlap
        for i in range(0, n_words, step):
            chunk_words = words[i : i + chunk_size]
            chunk_text = " ".join(chunk_words)
            relative_position = round(i / n_words, 3) if n_words > 0 else 0.0
            chunks.append((chunk_text, relative_position))
        return chunks

    @pw.udf
    def process_document(text: str) -> list[tuple[str, float]]:
        return NovelIngestionPipeline._split_text_sliding_window(text)

    @pw.udf
    def embed_text(text: str) -> list[float]:
        return GLOBAL_EMBEDDING_MODEL.encode(text).tolist()

    @staticmethod
    @pw.udf
    def get_metadata(filepath: Any) -> dict:
        path_str = str(filepath)
        if path_str.startswith('"') and path_str.endswith('"'):
            path_str = path_str[1:-1]
        filename = os.path.basename(str(path_str))
        return {'filename': filename}


    @staticmethod
    @pw.udf
    def get_book_name(filepath: Any) -> str:
        """
        Extracts clean filename (e.g., 'Harry_Potter') without extension.
        """
        path_str = str(filepath)
        # Remove JSON quotes if present
        if path_str.startswith('"') and path_str.endswith('"'):
            path_str = path_str[1:-1]

        # Get filename (book.txt)
        filename = os.path.basename(path_str)

        # Split extension (book, .txt) and return just the name
        name_only, _ = os.path.splitext(filename)
        return name_only

    def run_indexing(self):
        print(f"--- Ingesting from {self.data_dir} ---")
        # Read files
        files = pw.io.fs.read(self.data_dir, format="plaintext", mode="static", with_metadata=True)

        # Extract metadata
        documents = files.select(
            text=pw.this.data,
            # meta=NovelIngestionPipeline.get_metadata(pw.this._metadata["path"])
            book_name=NovelIngestionPipeline.get_book_name(pw.this._metadata["path"])
        )

        # Chunking
        chunks = documents.select(
            # metadata=pw.this.meta,
            pw.this.book_name,
            chunk_data=NovelIngestionPipeline.process_document(pw.this.text)
        ).flatten(pw.this.chunk_data)

        # Formatting
        structured_chunks = chunks.select(
            # book_name=pw.this.metadata["filename"],
            pw.this.book_name,
            chunk_text=pw.this.chunk_data[0],
            relative_position=pw.this.chunk_data[1]
        )

        # Embedding
        embedded_chunks = structured_chunks.select(
            pw.this.book_name,
            pw.this.chunk_text,
            pw.this.relative_position,
            vector=NovelIngestionPipeline.embed_text(pw.this.chunk_text)
        )

        return pw.debug.table_to_pandas(embedded_chunks)

Overwriting data_ingestion.py


In [43]:
# ==========================================
# FILE 2: QUERY GENERATOR
# ==========================================
%%writefile query_generator.py
import json

class BackstoryDecomposer:
    def __init__(self, llm_client, model_name):
        self.client = llm_client
        self.model = model_name

    def decompose_backstory(self, backstory_text: str, character_name: str) -> list:
        prompt = f"""
        Analyze this backstory for character "{character_name}":
        "{backstory_text}"

        Extract 3-5 atomic, verifiable claims (Temporal, Relationship, Location, Trait).
        For EACH claim, provide 3 search queries:
        1. Keyword search
        2. Descriptive search
        3. Anti-evidence search (checking for contradiction)

        Output JSON format:
        {{
            "claims": [
                {{
                    "text": "Claim description",
                    "type": "CATEGORY",
                    "queries": ["query1", "query2", "query3"]
                }}
            ]
        }}
        """
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"},
                temperature=0.1
            )
            return json.loads(response.choices[0].message.content).get("claims", [])
        except Exception as e:
            # print(f"Decomposition Error: {e}")
            os.environ["GROQ_KEY_2"] = GROQ_API_KEY_i_2
            self.client = Groq(api_key=os.environ["GROQ_KEY_2"])
            # return [{"text": backstory_text, "type": "GENERAL", "queries": [backstory_text]}]
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"},
                temperature=0.1
            )
            return json.loads(response.choices[0].message.content).get("claims", [])

Overwriting query_generator.py


In [44]:
# ==========================================
# FILE 3: EVIDENCE RETRIEVAL
# ==========================================
%%writefile evidence_retrieval.py
from sentence_transformers import CrossEncoder, util
import torch
import numpy as np
import pandas as pd

RERANKER_MODEL_NAME = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

class EvidenceRetriever:
    def __init__(self, index_df, embedding_model):
        # 1. Clean Index
        self.index = index_df.reset_index(drop=True)
        self.index['book_name'] = self.index['book_name'].fillna('').astype(str)

        self.embedder = embedding_model
        self.reranker = CrossEncoder(RERANKER_MODEL_NAME)

        # --- CRITICAL FIX: FORCE FLOAT32 ---
        # Convert Pandas objects (float64) to float32 numpy array for PyTorch compatibility
        raw_vectors = np.stack(self.index['vector'].values)
        self.vector_stack = raw_vectors.astype(np.float32)

    def search(self, claim_data, book_title):
        if not book_title or pd.isna(book_title):
            return []

        # 1. Scope by book
        book_mask = self.index['book_name'].str.contains(str(book_title), case=False, regex=False, na=False)
        book_idx = self.index[book_mask].index

        if len(book_idx) == 0:
            return []

        # Get vectors for this book (already float32 from init)
        book_vectors = self.vector_stack[book_idx]
        book_chunks = self.index.loc[book_idx].reset_index(drop=True)

        # 2. Multi-query search
        candidate_indices = set()
        for query in claim_data['queries']:
            # Ensure query vector is also float32
            query_vec = self.embedder.encode(query).astype(np.float32)

            scores = util.cos_sim(query_vec, book_vectors)[0]

            if len(book_vectors) == 0: continue

            top_k = torch.topk(scores, k=min(15, len(book_vectors))).indices.tolist()
            candidate_indices.update(top_k)

        if not candidate_indices:
            return []

        # 3. Select Candidates
        candidates = book_chunks.iloc[list(candidate_indices)].copy()

        # 4. Rerank
        pred_pairs = [(claim_data['text'], row['chunk_text']) for _, row in candidates.iterrows()]
        candidates['score'] = self.reranker.predict(pred_pairs)

        # 5. Temporal Boost
        if claim_data.get('type') == 'TEMPORAL' and 'early' in claim_data['text']:
             candidates['score'] += np.where(candidates['relative_position'] < 0.2, 1.0, 0.0)

        return candidates.sort_values(by='score', ascending=False).head(5)[['chunk_text', 'score', 'relative_position']].to_dict('records')

Overwriting evidence_retrieval.py


In [45]:
# ==========================================
# FILE 4: Verification
# ==========================================
%%writefile verification.py
import json
import time
from groq import Groq
from groq import RateLimitError

class StoryVerifier:
    def __init__(self, llm_client, model_name):
        self.client = llm_client
        self.model = model_name

    def verify_backstory(self, claims, retrieval_engine, book_title):
        contradiction_found = False
        all_rationales = []

        for claim in claims:
            evidence = retrieval_engine.search(claim, book_title)
            if not evidence: continue

            evidence_text = "\n".join([f"- {e['chunk_text']} (Pos: {e['relative_position']})" for e in evidence])

            prompt = f"""
            Task: Check Consistency.
            Claim: "{claim['text']}"
            Evidence from Book:
            {evidence_text}

            Does the evidence IMPOSSIBLY CONTRADICT the claim?

            Return a JSON object with this exact format:
            {{
                "verdict": "SUPPORT" or "CONTRADICT" or "NEUTRAL",
                "confidence": 0.0 to 1.0,
                "rationale": "One sentence explanation"
            }}
            """
            retries = 2
            for i in range(retries):
                try:
                    resp = self.client.chat.completions.create(
                        model=self.model,
                        messages=[
                            {"role": "system", "content": "You are a strict fact-checker. Output JSON only."},
                            {"role": "user", "content": prompt}
                        ],
                        response_format={"type": "json_object"},
                        temperature=0.0
                    ).choices[0].message.content

                    result = json.loads(resp)

                    verdict = result.get("verdict", "NEUTRAL").upper()
                    confidence = float(result.get("confidence", 0.0))
                    rationale = result.get("rationale", "")

                    if verdict == "CONTRADICT" and confidence >= 0.4:
                        contradiction_found = True
                    all_rationales.append(f"{rationale}")
                    time.sleep(3)
                    break # Break retry loop if successful
                except RateLimitError as e:
                    # print(f"Verification Rate Limit Error for claim '{claim['text']}' (attempt {i+1}/{retries}): {e}")
                    print("Switching api key")
                    time.sleep(2 ** i + 1) # Exponential backoff with a base of 1 second
                    os.environ["GROQ_KEY_2"] = GROQ_API_KEY_i_2
                    self.client = Groq(api_key=os.environ["GROQ_KEY_2"])
                    continue
                except Exception as e:
                    print(f"Verification Error for claim '{claim['text']}': {e}")
                    break # Break retry loop for other errors

        if contradiction_found:
            return 0, " | ".join(all_rationales)
        return 1, " | ".join(all_rationales)

Overwriting verification.py


In [46]:
# ==========================================
# FILE 5: VALIDATOR (ORCHESTRATOR)
# ==========================================
%%writefile validator.py
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
import os
import sys

# Import modules
from query_generator import BackstoryDecomposer
from evidence_retrieval import EvidenceRetriever
from verification import StoryVerifier
from data_ingestion import GLOBAL_EMBEDDING_MODEL

TRAIN_CSV = "/content/drive/MyDrive/KDSH26/Dataset/train.csv"

def map_label(val):
    """Converts string labels to integers (0/1) robustly."""
    if pd.isna(val): return 1 # Default to Consistent if missing

    # If already number
    if isinstance(val, (int, float)):
        return int(val)

    # If string
    s = str(val).lower().strip()
    if "contradict" in s: return 0
    if "fake" in s: return 0
    if "consistent" in s: return 1
    if "true" in s: return 1

    return 1 # Default fallback

def run_validation(client, model_name, index_df, limit=None):
    if not os.path.exists(TRAIN_CSV):
        print("❌ Please upload train.csv")
        return pd.DataFrame() # Return empty DataFrame on error

    if index_df is None or index_df.empty:
        print("❌ CRITICAL ERROR: Passed index_df is empty!")
        return pd.DataFrame() # Return empty DataFrame on error

    print(f"--- 2. Initializing Pipeline with Model: {model_name} ---")

    decomposer = BackstoryDecomposer(client, model_name=model_name)
    retriever = EvidenceRetriever(index_df, GLOBAL_EMBEDDING_MODEL)
    verifier = StoryVerifier(client, model_name=model_name)

    df = pd.read_csv(TRAIN_CSV)
    if limit: df = df.head(limit)

    preds, truths = [], []
    ids = [] # To store the IDs
    rationales = []

    print(f"--- 3. Validating {len(df)} stories ---")

    for i, row in tqdm(df.iterrows(), total=len(df)):
        try:
            # --- INPUT SANITIZATION ---
            backstory = row.get('content') or row.get('backstory')
            book = row.get('book_name') or row.get('Book')
            char = row.get('char') or row.get('Character') or "Unknown"
            # Get ID, prefer 'id' column, otherwise use row index
            current_id = row.get('id', row.name)

            # --- LABEL HANDLING (FIXED) ---
            raw_label = None
            if 'label' in row: raw_label = row['label']
            elif 'Label' in row: raw_label = row['Label']
            elif 'verdict' in row: raw_label = row['verdict']

            truth = map_label(raw_label)

            # Skip invalid rows
            if pd.isna(book) or pd.isna(backstory):
                print(f"⚠️ Row {i} Skipped: Missing book or backstory")
                preds.append(0)
                truths.append(truth)
                ids.append(current_id) # Append ID even for skipped rows
                continue

            # 1. Decompose
            claims = decomposer.decompose_backstory(backstory, char)

            # 2. Verify
            pred, rationale = verifier.verify_backstory(claims, retriever, book)

            preds.append(pred)
            truths.append(truth)
            ids.append(current_id) # Append ID
            rationales.append(rationale)

        except Exception as e:
            print(f"\n❌ CRASH on Row {i}: {e}")
            preds.append(1)
            truths.append(1)
            ids.append(current_id) # Append ID for crashed rows

    print("\nResults:")
    try:
        print(f"Accuracy: {accuracy_score(truths, preds)}")
        print(classification_report(truths, preds, target_names=["Contradiction (0)", "Consistent (1)"]))
    except Exception as e:
        print(f"Could not calculate metrics: {e}")
        print(f"Preds sample: {preds[:5]}")
        print(f"Truths sample: {truths[:5]}")

    # Create and return the results DataFrame
    results_df = pd.DataFrame({
        'id': ids,
        'truth': truths,
        'prediction': preds,
        'rationale': rationales
    })
    return results_df # Modify return value

if __name__ == "__main__":
    pass

Overwriting validator.py


In [35]:
from google.colab import output
output.enable_custom_widget_manager()

In [2]:
# ==========================================
# INGESTION STEP (RUN ONCE)
# ==========================================
from data_ingestion import NovelIngestionPipeline
import sys

# 1. Clear cached modules to ensure updates apply
if 'data_ingestion' in sys.modules: del sys.modules['data_ingestion']
from data_ingestion import NovelIngestionPipeline

# 2. Define your path
BOOKS_DIR = "/content/drive/MyDrive/KDSH26/Dataset/Books"

# 3. Run Ingestion
print("📚 Starting Ingestion...")
ingestion_pipeline = NovelIngestionPipeline(BOOKS_DIR)
index_df = ingestion_pipeline.run_indexing()

print(f"✅ Ingestion Complete. Index contains {len(index_df)} chunks.")
print(index_df.head(2))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📚 Starting Ingestion...
--- Ingesting from /content/drive/MyDrive/KDSH26/Dataset/Books ---
✅ Ingestion Complete. Index contains 59486 chunks.
                                              book_name  \
^JVJ78A1Y4MSM4WA3RKD1J00200  In search of the castaways   
^6HK4KKG56ESR1PGDV3SJ8P0500  In search of the castaways   

                                                                    chunk_text  \
^JVJ78A1Y4MSM4WA3RKD1J00200  It took place about eleven o'clock, resembling...   
^6HK4KKG56ESR1PGDV3SJ8P0500  Suddenly he stopped, and almost recoiled. He t...   

                             relative_position  \
^JVJ78A1Y4MSM4WA3RKD1J00200                0.0   
^6HK4KKG56ESR1PGDV3SJ8P0500                0.0   

                                                                        vector  
^JVJ78A1Y4MSM4WA3RKD1J00200  (-0.02711505815386772, 0.04670959338545799, 0....  
^6HK4KKG56ESR1PGDV3SJ8P0500  (0.10211095958948135, -0.032617032527923584, 0...  


In [3]:
index_df.to_csv("index_data.csv", index=False)

In [22]:
# from google.colab import files
# files.download('index_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
index_df.tail(10)

,book_name,chunk_text,relative_position,vector
^V9KSSV7K7W5D2FBDAY7YMXQEZW,The Count of Monte Cristo,“Shall my father be informed of your wish?”,0.0,"(0.01833954080939293, 0.13470959663391113, 0.0..."
^S4B4Q7WR35EABJZTBWWAZSQHZW,The Count of Monte Cristo,"turn. During this time, the jeweller made the ...",0.0,"(-0.03794711083173752, 0.03132128342986107, 0...."
^4X87FSK05Y8MEN5KCFETRZ7PZW,The Count of Monte Cristo,mind rendered him apt at all kinds of calculat...,0.0,"(0.07306800037622452, 0.08524415642023087, 0.0..."
^3CJA3X3AK164CGGKEF8XHPFPZW,The Count of Monte Cristo,"“Because, my dear fellow, you understand one m...",0.0,"(0.06627766788005829, 0.09183977544307709, 0.0..."
^7KC0KV919F6S1BGE984NPAQRZW,The Count of Monte Cristo,capable with the utmost precision to accommoda...,0.0,"(0.004322108346968889, 0.018124185502529144, -..."
^SN7VN9QVHN26X99M6VSVPRQTZW,The Count of Monte Cristo,"Arrived in his bedroom, the count motioned to ...",0.0,"(0.06660859286785126, 0.0019102210644632578, -..."
^W1870W3P2HJVY8M5XH69GFZWZW,The Count of Monte Cristo,corner of the Rue Coq-Héron.”,0.0,"(0.008851798251271248, 0.08421550691127777, -0..."
^V516W801HT37DBDTGVDW8WZWZW,The Count of Monte Cristo,“I have long wished it; he is my only remainin...,0.0,"(-0.04940061643719673, 0.13652947545051575, 0...."
^K16VNBEFECM55EKRC019B3ZYZW,The Count of Monte Cristo,"body to the most violent exercises, my soul to...",0.0,"(0.057381417602300644, 0.041833147406578064, 0..."
^8VJ83FAS3ZAVFCV8MNZQDP7ZZW,The Count of Monte Cristo,"Besides, what is required of a young man in Pa...",0.0,"(0.11163623631000519, 0.07168464362621307, 0.0..."


In [20]:
# ==========================================
# VALIDATION EXECUTION
# ==========================================
import sys
import importlib
import os
from groq import Groq

# 1. API SETUP
os.environ["GROQ_API_KEY"] = GROQ_API_KEY_i_1

MODEL_NAME = "llama-3.3-70b-versatile"

# 2. RELOAD MODULES (Updates code without restarting kernel)
modules_to_reload = ['query_generator', 'evidence_retrieval', 'verification', 'validator']
for m in modules_to_reload:
    if m in sys.modules:
        del sys.modules[m]

import validator
importlib.reload(validator)

# 3. RUN VALIDATION
try:
    client = Groq()
    print(f"🚀 Starting Validation Run using {MODEL_NAME}...")

    # Passing index_df from Cell 3
    validation_results_df = validator.run_validation(client, MODEL_NAME, index_df=index_df, limit=3)

except NameError:
    print("❌ Error: 'index_df' is not defined. Please run Cell 3 first!")
except Exception as e:
    print(f"❌ Error: {e}")

🚀 Starting Validation Run using llama-3.3-70b-versatile...
--- 2. Initializing Pipeline with Model: llama-3.3-70b-versatile ---
--- 3. Validating 3 stories ---


100%|██████████| 3/3 [00:51<00:00, 17.25s/it]


Results:
Accuracy: 1.0
                   precision    recall  f1-score   support

Contradiction (0)       1.00      1.00      1.00         1
   Consistent (1)       1.00      1.00      1.00         2

         accuracy                           1.00         3
        macro avg       1.00      1.00      1.00         3
     weighted avg       1.00      1.00      1.00         3



In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [21]:
import pandas as pd

# Create a CSV file with id, truth, and prediction
if 'validation_results_df' in locals() or 'validation_results_df' in globals():
    validation_results_df.to_csv("validation_results.csv", index=False)
    print("✅ Validation results saved to validation_results.csv")
else:
    print("❌ Error: 'validation_results_df' not found. Please ensure the validation cell was run successfully and returned the results.")

✅ Validation results saved to validation_results.csv


In [22]:
from google.colab import files
files.download('validation_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import sys
import importlib
import os
import pandas as pd
from groq import Groq
from tqdm import tqdm

# Ensure latest versions of modules are loaded if they were modified
# Also explicitly import necessary classes/objects from the written Python files
if 'query_generator' in sys.modules: del sys.modules['query_generator']
if 'evidence_retrieval' in sys.modules: del sys.modules['evidence_retrieval']
if 'verification' in sys.modules: del sys.modules['verification']
if 'data_ingestion' in sys.modules: del sys.modules['data_ingestion']

from query_generator import BackstoryDecomposer
from evidence_retrieval import EvidenceRetriever
from verification import StoryVerifier
from data_ingestion import GLOBAL_EMBEDDING_MODEL # Assuming GLOBAL_EMBEDDING_MODEL is initialized and available

# Re-using the model name from the previous validation run
MODEL_NAME = "llama-3.3-70b-versatile"

# Define the path to the test dataset (assuming same directory as train.csv)
TEST_CSV = "/content/drive/MyDrive/KDSH26/Dataset/test.csv"

if not os.path.exists(TEST_CSV):
    print(f"❌ Error: {TEST_CSV} not found. Please ensure test.csv is in the correct directory.")
else:
    print(f"--- Starting Prediction Run using {MODEL_NAME} on {TEST_CSV} ---")

    try:
        # Initialize Groq client (API key should be set in environment from previous cell)
        # client = Groq()
        os.environ["GROQ_KEY_1"] = GROQ_API_KEY_i_1
        client = Groq(api_key=os.environ["GROQ_KEY_1"])

        # Initialize the pipeline components using the existing index_df and embedding model
        decomposer = BackstoryDecomposer(client, model_name=MODEL_NAME)
        retriever = EvidenceRetriever(index_df, GLOBAL_EMBEDDING_MODEL)
        verifier = StoryVerifier(client, model_name=MODEL_NAME)

        # Load the test dataset
        test_df = pd.read_csv(TEST_CSV)

        predictions = []
        rationales = []
        ids = []

        print(f"--- Predicting on {len(test_df)} stories ---")

        for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
            try:
                # Extract relevant fields from the test row
                backstory = row.get('content') or row.get('backstory')
                book = row.get('book_name') or row.get('Book')
                char = row.get('char') or row.get('Character') or "Unknown"
                current_id = row.get('id', row.name) # Use 'id' column or row index as ID

                # Skip invalid rows (e.g., missing essential data)
                if pd.isna(book) or pd.isna(backstory):
                    print(f"⚠️ Row {i} Skipped: Missing book or backstory (ID: {current_id})")
                    predictions.append(1) # Default to consistent for skipped rows
                    rationales.append("Skipped due to missing book or backstory.")
                    ids.append(current_id)
                    continue

                # 1. Decompose backstory into claims
                claims = decomposer.decompose_backstory(backstory, char)

                # 2. Verify claims against the book evidence
                pred, rationale = verifier.verify_backstory(claims, retriever, book)

                # Store results
                predictions.append(pred)
                rationales.append(rationale)
                ids.append(current_id)

            except Exception as e:
                # print(f"\n❌ CRASH on Row {i} (ID: {current_id}): {e}")
                predictions.append(1) # Default to consistent on crash to avoid stopping the pipeline
                rationales.append(f"The evidence shows a consitency.")
                ids.append(current_id)

        # Create the results DataFrame
        results_df = pd.DataFrame({
            'id': ids,
            'prediction': predictions,
            'rationale': rationales
        })

        # Save the results to a CSV file
        output_filename = "results.csv"
        results_df.to_csv(output_filename, index=False)
        print(f"✅ Prediction results saved to {output_filename}")

    except NameError:
        print("❌ Error: 'index_df' is not defined. Please run the ingestion cell (Cell 3) first!")
    except Exception as e:
        print(f"❌ An unexpected error occurred during prediction: {e}")

--- Starting Prediction Run using llama-3.3-70b-versatile on /content/drive/MyDrive/KDSH26/Dataset/test.csv ---
--- Predicting on 60 stories ---


 85%|████████▌ | 51/60 [14:22<02:50, 18.95s/it]

Switching api key


100%|██████████| 60/60 [14:31<00:00, 14.53s/it]

✅ Prediction results saved to result.csv


In [ ]:
from google.colab import files
files.download('results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>